# Should you scale the target in regression tasks?

It's fairly well known that it's often use, and sometimes essential, to scale (eg normalize or standardize) the features (i.e. `X`) before training a predictive model on them.

The question of when, if everm one should _also_ scale the target, `y`, sometimes comes up.

I asked ChatGPT for its opinion, and it surprised me a bit because it almost seems to suggest that you should always scale the target:

> Scaling the target variable in a regression task can be beneficial in certain scenarios:
> 
> 1. When using regularization techniques: If you are using a regression algorithm with regularization, such as Lasso or Ridge regression, scaling the target variable can help ensure that the magnitude of the coefficients and the regularization penalty term are on a similar scale. This can lead to more stable and accurate models.
> 
> 2. When using neural networks: Neural networks often work better when both input features and target variables are on a similar scale, as it can help with gradient-based optimization and avoid issues like vanishing or exploding gradients. Scaling the target variable in this context can lead to faster convergence and better model performance.
> 
> 3. When comparing models with different target scales: If you are comparing models across different datasets with varying target variable scales, scaling the target variable can help make performance comparisons more consistent and meaningful.
> 
> 4. When using gradient-based optimization algorithms: Some machine learning algorithms, like support vector regression (SVR), use gradient-based optimization methods. Scaling the target variable can help improve the convergence of the optimization process.
> 
> 5. When the target variable has a large range or extreme values: In cases where the target variable has a large range or extreme values, scaling it can help stabilize the learning process and avoid potential issues with numerical stability or model performance.
> 
> However, it is important to remember that after training the model, you will need to reverse the scaling transformation to interpret the model's predictions in their original scale.
> 
> In summary, while not always necessary, scaling the target variable in a regression task can be beneficial in specific situations, such as when using regularization, neural networks, or gradient-based optimization algorithms.

And there's [this conversation](https://stats.stackexchange.com/questions/111467/is-it-necessary-to-scale-the-target-value-in-addition-to-scaling-features-for-re) on StackOverflow, with answers ranging from "No" to "Maybe" to "Yes".

So let's load some data and try some things!

### TLDR

Experimentation suggests that very large or very small target values make SVR and MLPRegressor very unstable and hard to train, even with hyperparameter search. (I probably didn't think hard enough about how to compensate for the value magnitudes, mostly just playing with a couple of arguments). In each case, scaling the target (standardizing) solved the problem.

As far as I can tell, scaling the target did not substantially affect the performance of LinearRegression (with or without L2 regularization, see below re L1), KNN regression, SGD regression (surprised me), or RandomForestRegressor.

For whatever reason, I cannot get Lasso (L1 regularization) to converge at all on my data, even when scaling the input and the output.

If you're getting unstable results from an SVR or MLP, I think the best strategy may be to just use another algorithm, rather than trying to find the right hyperparameters, or deal with the hassle of scaling the target. If hellbent on using a neural net or SVR, just scale the target.

## Load some data

In [1]:
import pandas as pd

df = pd.read_csv('https://geocomp.s3.amazonaws.com/data/MD-GR-NPOR-RHOB-DT4P-DT4S.txt',
                 names='MD-GR-NPOR-RHOB-DT4P-DT4S'.split('-'),
                )

df.describe()

,MD,GR,NPOR,RHOB,DT4P,DT4S
count,1196.000000,1196.000000,1196.000000,1196.000000,1196.000000,1196.000000
mean,2790.536000,67.651168,0.327833,2492.510252,279.854858,462.820733
std,345.399768,16.584102,0.073247,113.628417,25.897177,396.698555
min,2193.036000,20.533973,0.063613,2085.521075,174.242612,-3278.379200
25%,2491.786000,55.929214,0.282957,2449.629850,262.940034,464.979536
50%,2790.536000,72.154001,0.341788,2506.717670,284.703725,515.470562
75%,3089.286000,80.129819,0.381460,2566.703012,299.693629,555.194036
max,3388.036000,156.760897,0.540542,2948.923828,342.492104,590.466475


In [2]:
# Remove problem samples.
df = df[df['DT4S'] > 200]

# Make new targets.
df['DT4S_scaled'] = df['DT4S'] * 100
df['DT4S_huge'] = df['DT4S'] * 1e3
df['DT4S_tiny'] = df['DT4S'] / 1e6

In [3]:
from sklearn.linear_model import LinearRegression, Ridge, SGDRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import make_pipeline


# Make X and y.
X = df[['MD', 'GR', 'NPOR', 'RHOB', 'DT4P']].values
y = df['DT4S_tiny'].values

# Split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Linear regression

Should not make a difference if I scale the target or not.

In [4]:
# Make and fit pipeline.
regr = LinearRegression()
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9477605931787968

With scaled target:

In [5]:
regr = TransformedTargetRegressor(LinearRegression(), transformer=StandardScaler())
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9477605931787968

Now with regularization, using `Ridge()`. The idea is that if you use regularization then it might be a good idea to scale the target.

In [6]:
regr = Ridge()
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9478444519961955

With scaled target:

In [84]:
regr = TransformedTargetRegressor(Ridge(), transformer=StandardScaler())
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9478444519961955

Makes no difference in either case, i.e. regularization or not.

Now with `Lasso()`

In [7]:
from sklearn.linear_model import Lasso

regr = Lasso(alpha=0)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

/home/data1/bin/miniconda3/envs/redflag/lib/python3.10/site-packages/sklearn/base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
/home/data1/bin/miniconda3/envs/redflag/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
/home/data1/bin/miniconda3/envs/redflag/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.223e-07, tolerance: 3.498e-10 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented i

0.9477605931787968

In [8]:
from sklearn.linear_model import Lasso

regr = Lasso()
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-0.0015762114490540657

In [9]:
regr = TransformedTargetRegressor(Lasso(), transformer=StandardScaler())
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-0.0015762114490540657

`Lasso` does not converge at all and I don't know why.

## Random forest

Now with `RandomForestRegressor()`

In [10]:
from sklearn.ensemble import RandomForestRegressor

regr = RandomForestRegressor()
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9595482554402164

In [11]:
regr = TransformedTargetRegressor(RandomForestRegressor(), transformer=StandardScaler())
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9622020937029279

## Stochastic gradient descent

And with SGD -- tiny not affect, huge is bad.

In [12]:
regr = SGDRegressor(penalty=None)  # No regularization.
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9465742697231702

In [13]:
regr = SGDRegressor(alpha=0.01)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9450423441129018

In [14]:
regr = TransformedTargetRegressor(SGDRegressor(alpha=0.01), transformer=StandardScaler())
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9482629278774858

## Neural network

In [15]:
from sklearn.neural_network import MLPRegressor

regr = MLPRegressor(alpha=0,  max_iter=1000)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-170807.59684778232

In [16]:
regr = MLPRegressor(alpha=0.0001, max_iter=1000)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-138922.25067943643

In [17]:
regr = TransformedTargetRegressor(MLPRegressor(), transformer=StandardScaler())
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9600540359490983

💥 So this one does blow up.

## KNN

In [18]:
from sklearn.neighbors import KNeighborsRegressor

regr = KNeighborsRegressor(n_neighbors=50)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9248696037011152

In [19]:
regr = KNeighborsRegressor()
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9507415190568468

In [20]:
regr = TransformedTargetRegressor(KNeighborsRegressor(), transformer=StandardScaler())
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9507415190568468

## Support vector machine

And with SVR -- all affected

In [21]:
regr = SVR(C=1e12)  # Almost no regularization.
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-1.2176570605683525

In [22]:
regr = SVR(C=1)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-1.2176570605683525

In [23]:
regr = TransformedTargetRegressor(SVR(C=1), transformer=StandardScaler())
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

0.9628423809808271

## Question: can you always rescue things with (say) GridSearch

**Short answer: Maybe, if you know which params to change and really check all the cases (esp near edges!). But for SVR at least, tiny values seem to be very difficult to compensate for.**

In [24]:
regr = SVR(C=1)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-1.2176570605683525

In [26]:
from sklearn.model_selection import GridSearchCV
import numpy as np


grid = GridSearchCV(pipe, param_grid={'svr__epsilon': np.logspace(-6, 6, 13)}, cv=6, verbose=1)
grid.fit(X_train, y_train)

Fitting 6 folds for each of 13 candidates, totalling 78 fits


GridSearchCV(cv=6,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svr', SVR(C=1))]),
             param_grid={'svr__epsilon': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06])},
             verbose=1)

In [27]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, param_grid={'svr__C': np.logspace(-6, 6, 13),
                                      'svr__epsilon': np.logspace(-6, 6, 13)},
                    cv=6, verbose=1)
grid.fit(X_train, y_train)

Fitting 6 folds for each of 169 candidates, totalling 1014 fits


GridSearchCV(cv=6,
             estimator=Pipeline(steps=[('standardscaler', StandardScaler()),
                                       ('svr', SVR(C=1))]),
             param_grid={'svr__C': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
                         'svr__epsilon': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06])},
             verbose=1)

In [28]:
grid.best_params_

{'svr__C': 1e-06, 'svr__epsilon': 10000.0}

In [29]:
grid.score(X_test, y_test)

-1.2176570469087928

In [30]:
regr = SVR(C=1e6)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-1.2176570605683525

In [31]:
regr = SVR(C=1e9)
pipe = make_pipeline(StandardScaler(), regr)
pipe.fit(X_train, y_train)
pipe.score(X_test, y_test)

-1.2176570605683525

Conclusion: probably better to just use another algorithm rather than trying to resuce stability from SVR or MLP.